# 🏭 Intelligent System for Industrial Equipment Failure Diagnosis
## Interactive Dashboard

Use this dashboard to diagnose **Spindle Overheat** events in the CNC milling machine.

**Instructions:**
1. Adjust the **Sensor Readings** below (simulating live telemetry).
2. Click **Run Diagnosis** to trigger the Hybrid AI.
3. View the **Probabilistic Analysis** and **Action Recommendation**.

In [1]:
# === SYSTEM SETUP ===
import sys
import os
import warnings
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Add project root to path
sys.path.append(os.path.abspath(''))

# Import System Modules
from src.integration import run_real, run_demo
from src.utils import load_cfg, path_for

# Suppress warnings for clean UI
warnings.filterwarnings('ignore')

# Load Config
cfg = load_cfg()
sensors = cfg['bn'].get('sensors', [])

print("⏳ Loading Model and Data... Please wait.")

# 1. LOAD MODEL & DATA ONCE (Global Scope)
# We force_retrain=False to use the model trained via main.py
try:
    MODEL, TEST_DATA, _, CFG = run_real(evidence=None, debug=False, force_retrain=False, return_test_data=True)
    
    # Load Raw Telemetry for the "Real Fault" button (Sliders need floats, not 0/1)
    RAW_TELEMETRY = pd.read_csv(path_for(CFG, "telemetry"))
    
    print("✅ System Loaded. Ready for Diagnosis.")
    print(f"   Model Type: {type(MODEL).__name__}")
    print(f"   Test Samples available: {len(TEST_DATA)}")

except Exception as e:
    print(f"❌ Error loading system: {e}")
    print("⚠️ PLEASE RUN 'python main.py' FIRST to train the model!")
    MODEL = None

⏳ Loading Model and Data... Please wait.
✅ System Loaded. Ready for Diagnosis.
   Model Type: BayesianNetwork
   Test Samples available: 20736


In [ ]:
# === INTERFACE CONTROLS (FINAL) ===

style = {'description_width': 'initial'}

# Title
header = widgets.HTML("<h3>📡 Sensor Telemetry Input</h3>")

# 1. Define ALL possible sensors ranges
sensor_config = {
    "spindle_temp": (20.0, 120.0, 0.1),
    "vibration_rms": (0.0, 5.0, 0.01),
    "coolant_flow": (0.0, 10.0, 0.1),
    "ambient_temp": (15.0, 40.0, 0.1),
    "feed_rate": (0.0, 5.0, 0.1),
    "spindle_speed": (0, 5000, 100),
    "load_pct": (0.0, 1.0, 0.01),
    "power_kw": (0.0, 10.0, 0.1),
    "tool_wear": (0.0, 0.5, 0.001)
}

active_sensors = cfg['bn'].get('sensors', [])
sensor_widgets = {}

# 2. Create Sliders
for s, (min_v, max_v, step) in sensor_config.items():
    label = s.replace('_', ' ').title()
    if s in active_sensors:
        label = f"⚡ {label}"
    
    if isinstance(min_v, int):
        w = widgets.IntSlider(value=min_v, min=min_v, max=max_v, step=step, description=label, style=style, layout=widgets.Layout(width='95%'))
    else:
        w = widgets.FloatSlider(value=min_v, min=min_v, max=max_v, step=step, description=label, style=style, layout=widgets.Layout(width='95%'))
    sensor_widgets[s] = w

keys = list(sensor_widgets.keys())
half = (len(keys) + 1) // 2
left_box = widgets.VBox([sensor_widgets[k] for k in keys[:half]])
right_box = widgets.VBox([sensor_widgets[k] for k in keys[half:]])
ui_inputs = widgets.HBox([left_box, right_box])

# --- ACTION BUTTONS ---
btn_diagnose = widgets.Button(
    description='🔍 Run Diagnosis',
    button_style='primary',
    layout=widgets.Layout(width='180px'),
    icon='check'
)

# SINGLE RANDOM BUTTON
btn_random = widgets.Button(
    description='🎲 Random Scenario',
    button_style='info',
    layout=widgets.Layout(width='180px'),
    icon='random'
)

output_area = widgets.Output()

# === LOGIC ===

def on_diagnose_click(b):
    with output_area:
        clear_output()
        if MODEL is None:
            print("❌ Model not loaded. Run main.py first.")
            return

        evidence = {s: w.value for s, w in sensor_widgets.items()}
        
        try:
            result = run_demo(evidence, debug=False, model_override=MODEL)
            render_dashboard(result, evidence)
        except Exception as e:
            print(f"❌ Error: {e}")
def on_random_click(b):
    """Loads ANY sample (Normal or Fault) from history."""
    if TEST_DATA is None: return

    # 1. Pick ANY index from the test set
    indices = TEST_DATA.index
    if len(indices) > 0:
        idx = np.random.choice(indices)
        
        # 2. Get RAW values
        if 'RAW_TELEMETRY' in globals():
            raw_row = RAW_TELEMETRY.loc[idx]
            for s, w in sensor_widgets.items():
                if s in raw_row:
                    val = raw_row[s]
                    if val < w.min: val = w.min
                    if val > w.max: val = w.max
                    w.value = val
        
        # 3. Auto-run diagnosis
        on_diagnose_click(None)
    

def render_dashboard(res, evidence):
    p_oh = res['p_overheat']
    cause = res['top_cause']
    action = res['recommended_action']
    costs = res.get('expected_costs', {})
    chosen_cost = costs.get(action, 0.0)
    
    color = "#28a745" # Green
    status = "NORMAL"
    if p_oh > 0.3: 
        color = "#ffc107" # Orange
        status = "WARNING"
    if p_oh > 0.7:
        color = "#dc3545" # Red
        status = "CRITICAL"
        
    procs = res.get('procedures', [])
    proc_html = "<ul>" + "".join([f"<li>{p}</li>" for p in procs]) + "</ul>" if procs else "None"

    html = f"""
    <div style='border: 2px solid {color}; border-radius: 10px; padding: 20px; background-color: #ffffff; box-shadow: 0 4px 8px 0 rgba(0,0,0,0.1); font-family: sans-serif;'>
        <h2 style='color: {color}; margin-top: 0; border-bottom: 1px solid #eee; padding-bottom: 10px;'>
            SYSTEM STATUS: {status}
        </h2>
        
        <div style='display: flex; flex-wrap: wrap; justify-content: space-between;'>
            <div style='flex: 1; min-width: 300px; padding-right: 20px;'>
                <h4 style='color: #007bff; border-bottom: 2px solid #007bff; display: inline-block;'>🔍 Bayesian Inference</h4>
                <div style='background: #f8f9fa; padding: 15px; border-radius: 8px; margin-top: 10px;'>
                    <div style='font-size: 1.2em; margin-bottom: 8px;'>Overheat Probability: <b style='font-size: 1.3em; color: {color}'>{p_oh:.1%}</b></div>
                    <div style='font-size: 1.2em;'>Most Likely Cause: <b>{cause}</b></div>
                    <div style='margin-top: 5px; font-size: 0.9em; color: #666;'>
                        (Confidence: {res['probabilities'].get(cause, 0):.1%})
                    </div>
                </div>
            </div>
            
            <div style='flex: 1; min-width: 300px; border-left: 1px solid #eee; padding-left: 20px;'>
                <h4 style='color: #28a745; border-bottom: 2px solid #28a745; display: inline-block;'>🛠️ Decision Support</h4>
                <div style='font-size: 1.4em; font-weight: bold; color: #333; margin: 10px 0;'>
                    ➤ {action}
                </div>
                <table style='width:100%; font-size: 0.9em; border-collapse: collapse;'>
                    <tr style='border-bottom: 1px solid #eee;'>
                        <td style='padding: 8px 0; color: #555;'><b>Expected Cost:</b></td>
                        <td style='padding: 8px 0; font-weight: bold;'>{chosen_cost:.2f} €</td>
                    </tr>
                    <tr>
                        <td style='padding: 8px 0; vertical-align: top; color: #555;'><b>Procedures:</b></td>
                        <td style='padding: 8px 0;'>{proc_html}</td>
                    </tr>
                </table>
            </div>
        </div>
    </div>
    """
    display(HTML(html))
    
    # Hide Root Cause chart if normal to keep UI clean
    if p_oh > 0.1 and 'probabilities' in res:
        display(HTML("<h4 style='margin-top: 20px; color: #666;'>📊 Root Cause Probability Distribution</h4>"))
        for c, p in res['probabilities'].items():
            bar_width = int(p * 100)
            bar_color = "#dc3545" if c == cause and p > 0.5 else "#17a2b8"
            display(HTML(f"""
                <div style='margin-bottom: 8px; display: flex; align-items: center; font-family: monospace;'>
                    <div style='width: 160px; font-weight: bold;'>{c}</div>
                    <div style='width: 200px; background: #e9ecef; border-radius: 4px; margin-right: 10px; height: 12px;'>
                        <div style='width: {p*100}%; background-color: {bar_color}; height: 100%; border-radius: 4px;'></div>
                    </div>
                    <div style='width: 50px; text-align: right;'>{p:.1%}</div>
                </div>
            """))

btn_diagnose.on_click(on_diagnose_click)
btn_random.on_click(on_random_click)

# Display
display(header, ui_inputs, widgets.HBox([btn_diagnose, btn_random]), output_area)

HTML(value='<h3>📡 Sensor Telemetry Input</h3>')

Output()